## 02 - Scraping the HTML

In [2]:
import re
import requests
from bs4 import BeautifulSoup

In [103]:
#with open('output1.html', 'r') as f:

    #contents = f.read()

    #doc = BeautifulSoup(contents, 'lxml')

In [104]:
#Grab the HTML that contains the nominations. 

#results = doc.select('#main')
#results


In [115]:
list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

nominations = []

# Loop through each html-file and extract the nominations.
for i in list:
    with open(f'output{i}.html', 'r') as f:

        contents = f.read()

        doc = BeautifulSoup(contents, 'lxml')

    #Grab the HTML that contains the nominations. 

    results = doc.select('#main')

    #And then I create a variable (a list) which contains each nomination.

    elements = results[0].select('.expanded')

    for element in elements:
        nomination = {}
        nomination['name'] = element.strong.text
        # And remove "— Department of Justice" or " — The Judiciary" from the names
        nomination['name']= nomination['name'].replace(' — Department of Justice', '')
        nomination['name']= nomination['name'].replace(' — The Judiciary', '')
    
        # And then I grab all the nomination numbers. 
        nomination['nomination_no'] = element.a.next
    
        nomination['congress_no'] = element.strong.previous.strip()
        # And remove the extra – and (year-year)
        nomination['congress_no']= nomination['congress_no'].replace('—', '').strip()
    
    
        # Here I grab the information on the circuit for the nominations for Circuit Court judges. All the rest of the nominations are passed. 
        if "Circuit" in element.find(class_="result-item").text.strip():
            nomination['post'] = element.find(class_="result-item").text.strip()
        else:
            pass
    
        # Here I grab the information on the votes and the day of the confirmation but only for the nominees that have been confirmed by Senate.
        if "Confirmed" in element.find_all(class_="result-item")[-1].text.strip():
            nomination['vote'] = element.find_all(class_="result-item")[-1].text.strip()
        else:
            pass
    
        #print(nomination)
        nominations.append(nomination)
        
#print(nominations)

In [112]:
import pandas as pd
df = pd.DataFrame(nominations)
df

,name,nomination_no,congress_no,post,vote
0,Ismail J. Ramsey,PN2797,117th Congress (2021-2022),NaN,NaN
1,Kate E. Brubacher,PN2796,117th Congress (2021-2022),NaN,NaN
2,Justin L. Martinez,PN2735,117th Congress (2021-2022),NaN,NaN
3,Stephen K. Eberle,PN2734,117th Congress (2021-2022),NaN,NaN
4,Scott Winston Colom,PN2733,117th Congress (2021-2022),NaN,NaN
...,...,...,...,...,...
2851,Kathleen McCree Lewis,PN5,107th Congress (2001-2002),"Kathleen McCree Lewis, of Michigan, to be Unit...",NaN
2852,Roger L. Gregory,PN4,107th Congress (2001-2002),"Roger L. Gregory, of Virginia, to be United St...",NaN
2853,Barry P. Goode,PN3,107th Congress (2001-2002),"Barry P. Goode, of California, to be United St...",NaN
2854,James E. Duffy Jr.,PN2,107th Congress (2001-2002),"James E. Duffy, Jr., of Hawaii, to be United S...",NaN


In [116]:
# Drop all the rows that where post or vote is NaN. Those are the entries where the nomination isn't for a circuit judge
# or where the nomination has not been confirmed by Congress. 
df1 = df.dropna(subset=['post','vote']) 

In [118]:
# And then I save the data to a csv-file.

df1.to_csv("nominations.csv", index=False)